In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import lime
from lime import lime_image
from lime import submodular_pick
import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
import sys
sys.path.append('..')
from dataset import batcher

In [49]:
MODEL_DIR = '../saved_models/'
LR_RATE = 0.001
OPT = tf.keras.optimizers.Adam(learning_rate=LR_RATE)
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']
COUNTRIES = ['ethiopia-2018', 'nigeria-2018', 'malawi-2016', 'pollution-2018']

In [71]:
# Get test split
bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SW_IR1', 'SW_IR2', 'TEMP', 'VIIRS', 'DELTA_TEMP', 'CO']
# bands = ['VIIRS']
test_batcher = batcher.Batcher(bands, country_year="ethiopia-2018", bucket=False, shuffle=False, split='test', repeat=1, batch_size=3000).get_dataset()
x_true = []
y_true = []
for x, y in test_batcher:
    x_true.append(x)
    y_true.append(y)

print(np.shape(x_true))

(1, 652, 224, 224, 10)


In [72]:
# Load model
experiments = ['imagery_sample_vgg_regression_ALL']

def load_model_from_dir(model_name):
    model_path = MODEL_DIR + model_name + '.h5'
    model = tf.keras.models.load_model(model_path, compile=False)
    
    return model

In [73]:
# Get consumption predictions
trained_model = load_model_from_dir(experiments[0])
trained_model.compile(
            loss=LOSS,
            optimizer=OPT,
            metrics=METRICS,
        )

y_preds = trained_model.predict_on_batch(x_true)
print(y_preds)

2023-02-26 11:36:24.622640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


[[ 8.29432383e-02]
 [-1.00976616e-01]
 [-1.00780278e-01]
 [ 3.58850509e-02]
 [ 1.37671456e-01]
 [-9.18138176e-02]
 [ 4.66265708e-01]
 [ 3.16535264e-01]
 [ 7.64195085e-01]
 [ 1.43512294e-01]
 [ 3.63314986e-01]
 [ 4.44799572e-01]
 [-1.08663619e-01]
 [-1.55623525e-01]
 [ 2.79339761e-01]
 [-1.29014879e-01]
 [ 1.56375006e-01]
 [ 2.83457011e-01]
 [-5.80678880e-02]
 [ 5.27194794e-03]
 [-1.65546417e-01]
 [ 1.59544736e-01]
 [-9.15615559e-02]
 [ 9.41421911e-02]
 [-5.80678880e-02]
 [ 5.21213293e-01]
 [-1.27955705e-01]
 [-1.08663619e-01]
 [ 5.88308811e-01]
 [-7.39860237e-02]
 [ 5.15910089e-01]
 [ 1.01270586e-01]
 [ 2.93263525e-01]
 [-6.06132597e-02]
 [ 8.29046126e-03]
 [ 2.86984831e-01]
 [ 5.34203555e-03]
 [-1.32642537e-01]
 [-7.27555379e-02]
 [ 4.94348139e-01]
 [ 9.11262929e-02]
 [-3.01012341e-02]
 [-5.78183457e-02]
 [ 3.54335099e-01]
 [-9.44361538e-02]
 [ 1.28349558e-01]
 [-1.25506381e-02]
 [ 2.67370224e-01]
 [ 4.51466739e-02]
 [ 3.78083557e-01]
 [ 4.18093234e-01]
 [ 4.55212921e-01]
 [-1.4467689

In [80]:
y_true

array([ 0.7888055 ,  1.1076248 ,  2.621244  ,  2.7076216 ,  3.2089376 ,
        3.374914  ,  4.481263  ,  1.9446048 ,  6.2642536 ,  2.7563071 ,
        0.8707002 ,  4.701411  ,  1.7109228 ,  2.159856  ,  6.025476  ,
        5.635698  ,  1.7794498 ,  2.184468  ,  3.1767793 ,  3.9774497 ,
       15.425267  ,  1.9981123 ,  3.209867  ,  1.6324995 ,  2.6567554 ,
       24.176773  ,  7.369581  ,  3.1434088 ,  4.23116   ,  1.5137556 ,
        4.8134484 , 12.403935  ,  2.8116813 ,  8.322237  ,  3.238693  ,
        2.027193  ,  1.2307045 ,  3.068319  ,  2.3052027 ,  3.6615393 ,
        1.7056962 ,  2.7531648 ,  1.5288144 ,  6.8396215 ,  1.3472768 ,
        3.2301898 ,  3.9976203 ,  6.2659116 ,  2.6047673 ,  2.7142165 ,
        2.9079616 , 11.474877  ,  1.1566951 ,  1.4518763 ,  6.203923  ,
        3.3771734 ,  3.786213  ,  7.9425907 ,  2.381878  ,  9.53462   ,
        1.7402346 ,  4.9096785 ,  4.6480675 ,  6.7485523 ,  7.3544044 ,
        1.9378757 ,  2.6118805 ,  4.20326   ,  1.2509547 ,  3.52

In [75]:
# Find r-squared
y_true = np.squeeze(y_true)
y_preds = np.squeeze(y_preds)
y_preds = y_preds

# y_true = 2 * ((y_true - min(y_true)) / (max(y_true) - min(y_true))) - 1
# y_preds = 2 * ((y_preds - min(y_preds)) / (max(y_preds) - min(y_preds))) - 1

r2 = pearsonr(y_true, y_preds)[0]

In [76]:
cons_diffs = y_true - y_preds

In [77]:
def find_closest_index(arr, val):
    idx = np.abs(arr - val).argmin()
    return idx

In [81]:
idx = find_closest_index(cons_diffs, 0)
print(idx)
print(y_true[idx])

220
0.2678503
